In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
import eli5
import csv

In [5]:
train= pd.read_csv(r'MoviesWithSynopsisAndCategory.csv').sample(frac=0.7)
test= pd.read_csv(r'MoviesWithSynopsisAndCategory.csv').sample(frac=0.3)

In [6]:
text_transformer = TfidfVectorizer( ngram_range=(1, 2),  max_features=150000)

In [7]:
%%time
X_train_text = text_transformer.fit_transform(train['ملخص'])
X_test_text = text_transformer.transform(test['ملخص'])

Wall time: 261 ms


In [8]:
X_train_text.shape,X_test_text.shape

((828, 43400), (355, 43400))

In [9]:
logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)

In [10]:
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=17)


In [11]:
%%time
cv_results = cross_val_score(logit, X_train_text, train['تصنيف الفيلم'], cv=skf, scoring='f1_micro')

C:\Users\MCST\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(("The least populated class in y has only %d"


Wall time: 26.9 s


In [12]:
cv_results, cv_results.mean()


(array([0.48067633, 0.47826087]), 0.47946859903381644)

In [13]:
%%time
logit.fit(X_train_text, train['تصنيف الفيلم'])

Wall time: 15.1 s


LogisticRegression(C=50.0, multi_class='multinomial', n_jobs=4, random_state=17)

In [14]:
%%time
eli5.show_weights(estimator=logit, 
                  feature_names= list(text_transformer.get_feature_names()),
                 top=(50, 5))

Wall time: 318 ms


In [15]:
test_preds = logit.predict(X_test_text)
pd.DataFrame(test_preds, columns=['تصنيف'])


,تصنيف
0,رومانسي
1,دراما
2,دراما
3,كوميدي
4,دراما
...,...
350,رومانسي
351,دراما
352,كوميدي
353,كوميدي


In [17]:
count=0
a=list(test_preds)
b=list(test["تصنيف الفيلم"])
for i in range(len(test)):
    if a[i]==b[i]:
        count+=1
print("Accuracy Precentage ="+str(round((count/len(test))*100,3)))

Accuracy Precentage =84.225
